In [626]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [627]:
df=df = pd.read_csv('StudentDepressionDataset.csv')

In [628]:
df.sample(3)

,id,Gender,Age,City,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
24325,122459,Female,23.0,Nashik,Student,4.0,0.0,7.02,1.0,0.0,Less than 5 hours,Moderate,M.Tech,No,11.0,4.0,Yes,1
22809,115030,Female,24.0,Agra,Student,5.0,0.0,8.90,4.0,0.0,Less than 5 hours,Moderate,Class 12,Yes,8.0,1.0,Yes,1
26760,135022,Male,28.0,Vasai-Virar,Student,1.0,0.0,8.28,1.0,0.0,Less than 5 hours,Healthy,BHM,Yes,7.0,2.0,Yes,1


In [629]:
# remove id column
df.drop('id', axis=1, inplace=True)



In [630]:
df.shape

(27901, 17)

In [631]:
df.nunique()

Gender                                     2
Age                                       34
City                                      52
Profession                                14
Academic Pressure                          6
Work Pressure                              3
CGPA                                     332
Study Satisfaction                         6
Job Satisfaction                           5
Sleep Duration                             5
Dietary Habits                             4
Degree                                    28
Have you ever had suicidal thoughts ?      2
Work/Study Hours                          13
Financial Stress                           5
Family History of Mental Illness           2
Depression                                 2
dtype: int64

In [632]:
missing_values_count = df.isnull().sum()
print("Number of Missing Values in Each Column:\n", missing_values_count)

Number of Missing Values in Each Column:
 Gender                                   0
Age                                      0
City                                     0
Profession                               0
Academic Pressure                        0
Work Pressure                            0
CGPA                                     0
Study Satisfaction                       0
Job Satisfaction                         0
Sleep Duration                           0
Dietary Habits                           0
Degree                                   0
Have you ever had suicidal thoughts ?    0
Work/Study Hours                         0
Financial Stress                         3
Family History of Mental Illness         0
Depression                               0
dtype: int64


In [633]:
# make missing values of financial stress to be the mean of the column
df['Financial Stress'].fillna(df['Financial Stress'].mean(), inplace=True)
df.isnull().sum()


C:\Users\aslam\AppData\Local\Temp\ipykernel_33520\3258471088.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Financial Stress'].fillna(df['Financial Stress'].mean(), inplace=True)


Gender                                   0
Age                                      0
City                                     0
Profession                               0
Academic Pressure                        0
Work Pressure                            0
CGPA                                     0
Study Satisfaction                       0
Job Satisfaction                         0
Sleep Duration                           0
Dietary Habits                           0
Degree                                   0
Have you ever had suicidal thoughts ?    0
Work/Study Hours                         0
Financial Stress                         0
Family History of Mental Illness         0
Depression                               0
dtype: int64

In [634]:
df.shape

(27901, 17)

In [635]:
df['Financial Stress'].value_counts()

Financial Stress
5.000000    6715
4.000000    5775
3.000000    5226
1.000000    5121
2.000000    5061
3.139867       3
Name: count, dtype: int64

In [636]:
# removing rows with values of age appearing less than or equal to 4 times
df = df[df['Financial Stress'].map(df['Financial Stress'].value_counts()) >=4]

In [637]:
df.shape

(27898, 17)

In [638]:
# job satisfaction is mostly 0, so we can drop it, as it will not be useful
# profession is mostly student, so we can drop it, as it will not be useful
# work pressure is mostly 0, so we can drop it, as it will not be useful
df.drop(['Job Satisfaction', 'Profession','Work Pressure'], axis=1, inplace=True)


In [639]:
df.shape

(27898, 14)

In [640]:
#  Remove Rows with 'Others' in Sleep Duration, Dietary Habits, Degree
df = df[~df['Sleep Duration'].str.contains('Others', na=False)]
df = df[~df['Dietary Habits'].str.contains('Others', na=False)]
df = df[~df['Degree'].str.contains('Others', na=False)]

#  Remove Rows with Study Satisfaction Value Equal to 0
df = df[df['Study Satisfaction'] != 0]


In [641]:
df.shape

(27823, 14)

In [642]:
df.nunique()

Gender                                     2
Age                                       34
City                                      52
Academic Pressure                          6
CGPA                                     332
Study Satisfaction                         5
Sleep Duration                             4
Dietary Habits                             3
Degree                                    27
Have you ever had suicidal thoughts ?      2
Work/Study Hours                          13
Financial Stress                           5
Family History of Mental Illness           2
Depression                                 2
dtype: int64

In [643]:
df['Age'].value_counts()

Age
24.0    2254
20.0    2230
28.0    2126
29.0    1946
33.0    1890
25.0    1781
21.0    1724
23.0    1638
18.0    1581
19.0    1559
34.0    1464
27.0    1457
31.0    1423
32.0    1255
22.0    1157
26.0    1152
30.0    1138
35.0      10
38.0       7
36.0       7
42.0       4
39.0       3
48.0       3
46.0       2
43.0       2
37.0       2
44.0       1
51.0       1
49.0       1
58.0       1
54.0       1
59.0       1
56.0       1
41.0       1
Name: count, dtype: int64

In [644]:
# removing rows with values of age appearing less than or equal to 10 times
df = df[df['Age'].map(df['Age'].value_counts()) >= 11]


In [645]:
df.shape

(27775, 14)

In [646]:
df.nunique()

Gender                                     2
Age                                       17
City                                      52
Academic Pressure                          6
CGPA                                     331
Study Satisfaction                         5
Sleep Duration                             4
Dietary Habits                             3
Degree                                    27
Have you ever had suicidal thoughts ?      2
Work/Study Hours                          13
Financial Stress                           5
Family History of Mental Illness           2
Depression                                 2
dtype: int64

In [647]:
df['City'].value_counts()

City
Kalyan                1565
Srinagar              1365
Hyderabad             1335
Vasai-Virar           1286
Lucknow               1150
Thane                 1138
Ludhiana              1106
Agra                  1089
Surat                 1076
Kolkata               1055
Jaipur                1035
Patna                  999
Pune                   966
Visakhapatnam          964
Ahmedabad              944
Bhopal                 929
Chennai                883
Meerut                 820
Rajkot                 810
Bangalore              765
Delhi                  765
Ghaziabad              741
Mumbai                 694
Vadodara               690
Varanasi               682
Nagpur                 646
Indore                 642
Kanpur                 607
Nashik                 544
Faridabad              458
Saanvi                   2
Bhavna                   2
City                     2
Harsha                   2
Less Delhi               1
M.Tech                   1
3.0                    

In [648]:
# removing rows with values of City appearing less than or equal to 10 times
df = df[df['City'].map(df['City'].value_counts()) >= 3]

In [649]:
df.shape

(27749, 14)

In [650]:
# there is too  many degrees, so we can group them into 3 categories
df['Degree'].value_counts()


Degree
Class 12    6054
B.Ed        1859
B.Com       1501
B.Arch      1474
BCA         1429
MSc         1185
B.Tech      1151
MCA         1037
M.Tech      1018
BHM          923
BSc          882
M.Ed         815
B.Pharm      809
M.Com        731
BBA          696
MBBS         689
LLB          668
BE           609
BA           595
M.Pharm      580
MD           569
MBA          560
MA           544
PhD          515
LLM          481
MHM          191
ME           184
Name: count, dtype: int64

In [651]:
# we can see that there are many degrees, so we can group them into 3 categories for better encoding later 

# making class 12th as higher secondary
df['Degree'] = df['Degree'].replace('Class 12th', 'Higher Secondary')


In [652]:
# making bachelors
bachelors_replacements = {
    'B.Ed': 'Bachelor',
    'B.Com': 'Bachelor',
    'B.Pharm': 'Bachelor',
    'BCA': 'Bachelor',
    'BE': 'Bachelor',
    'BHM': 'Bachelor',
    'MBBS': 'Bachelor', 
    'LLB': 'Bachelor',
    'BA': 'Bachelor',
    'B.Arch': 'Bachelor',
    'BBA': 'Bachelor',
    'B.Tech': 'Bachelor',
    'BSc': 'Bachelor',
}

# Apply replacements
df['Degree'] = df['Degree'].replace(bachelors_replacements)

In [653]:
# making masters
master_replacements = {
    'M.Tech': 'Master',
    'MCA': 'Master',
    'MBA': 'Master',
    'MSc': 'Master',
    'M.Com': 'Master',
    'MA': 'Master',
    'MHM': 'Master',
    'M.Arch': 'Master',
    'M.Pharm': 'Master',
    'ME': 'Master',  
    'LLM': 'Master',
    'MD': 'Master',  
    'M.Ed': 'Master',
    'PhD': 'Master' 
}

# Apply replacements
df['Degree'] = df['Degree'].replace(master_replacements)



In [654]:
df.sample(3)

,Gender,Age,City,Academic Pressure,CGPA,Study Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
26671,Male,28.0,Ludhiana,5.0,9.97,1.0,Less than 5 hours,Unhealthy,Master,Yes,11.0,4.0,No,1
22533,Male,24.0,Srinagar,3.0,7.91,2.0,Less than 5 hours,Healthy,Bachelor,No,1.0,2.0,Yes,0
14267,Female,24.0,Indore,5.0,9.31,3.0,Less than 5 hours,Unhealthy,Master,Yes,12.0,5.0,Yes,1


In [655]:
# doing one hot encoding for the categorical variables
df = pd.get_dummies(df, columns=['City', 'Degree', 'Sleep Duration', 'Dietary Habits'])


In [656]:
df.shape

(27749, 50)

In [657]:
df.info ()

<class 'pandas.core.frame.DataFrame'>
Index: 27749 entries, 0 to 27900
Data columns (total 50 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Gender                                 27749 non-null  object 
 1   Age                                    27749 non-null  float64
 2   Academic Pressure                      27749 non-null  float64
 3   CGPA                                   27749 non-null  float64
 4   Study Satisfaction                     27749 non-null  float64
 5   Have you ever had suicidal thoughts ?  27749 non-null  object 
 6   Work/Study Hours                       27749 non-null  float64
 7   Financial Stress                       27749 non-null  float64
 8   Family History of Mental Illness       27749 non-null  object 
 9   Depression                             27749 non-null  int64  
 10  City_Agra                              27749 non-null  bool   
 11  City_Ah

In [658]:
# mental illness,sucidial thoughts, and gender are objects but only got 2 answers so we can convert them to binary
df['Family History of Mental Illness'] = df['Family History of Mental Illness'].map({'Yes': 1, 'No': 0})
df['Have you ever had suicidal thoughts ?'] = df['Have you ever had suicidal thoughts ?'].map({'Yes': 1, 'No': 0})
df['Gender']=df['Gender'].map({'Male':1 , 'Female':0})



In [659]:
# split data into X and y
X=df.drop('Depression', axis=1)
y=df['Depression']


In [660]:
# split data into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [661]:
# logistic regression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

LR=LogisticRegression(solver='lbfgs', max_iter=1000)
LR.fit(X_train, y_train)
y_pred_LR=LR.predict(X_test)
accuracy=accuracy_score(y_test, y_pred_LR)
print('Accuracy:', accuracy)



# mse
from sklearn.metrics import mean_squared_error
mse_LR=mean_squared_error(y_test, y_pred_LR)
print('Mean Squared Error:', mse_LR)


# mae
from sklearn.metrics import mean_absolute_error
mae_LR=mean_absolute_error(y_test, y_pred_LR)
print('Mean Absolute Error:', mae_LR)


Accuracy: 0.8367567567567568
Mean Squared Error: 0.16324324324324324
Mean Absolute Error: 0.16324324324324324


In [662]:
# linear discriminant analysis
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score

LDA=LinearDiscriminantAnalysis()
LDA.fit(X_train, y_train)
y_pred_LDA=LDA.predict(X_test)
accuracy=accuracy_score(y_test, y_pred_LDA)
print('Accuracy:', accuracy)




# mse
from sklearn.metrics import mean_squared_error
mse_LDA=mean_squared_error(y_test, y_pred_LDA)
print('Mean Squared Error:', mse_LDA)

# mae
from sklearn.metrics import mean_absolute_error
mae_LDA=mean_absolute_error(y_test, y_pred_LDA)
print('Mean Absolute Error:', mae_LDA)


Accuracy: 0.8369369369369369
Mean Squared Error: 0.16306306306306306
Mean Absolute Error: 0.16306306306306306


In [663]:
# k neighbors classifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

KNN=KNeighborsClassifier()
KNN.fit(X_train, y_train)
y_pred_KNN=KNN.predict(X_test)
accuracy=accuracy_score(y_test, y_pred_KNN)
print('Accuracy:', accuracy)

# mse
from sklearn.metrics import mean_squared_error
mse_KNN=mean_squared_error(y_test, y_pred_KNN)
print('Mean Squared Error:', mse_KNN)

# mae
from sklearn.metrics import mean_absolute_error
mae_KNN=mean_absolute_error(y_test, y_pred_KNN)
print('Mean Absolute Error:', mae_KNN)



Accuracy: 0.7972972972972973
Mean Squared Error: 0.20270270270270271
Mean Absolute Error: 0.20270270270270271


In [664]:
# decision tree classifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

DTC=DecisionTreeClassifier()
DTC.fit(X_train, y_train)
y_pred_DTC=DTC.predict(X_test)
accuracy=accuracy_score(y_test, y_pred_DTC)
print('Accuracy:', accuracy)

# mae
from sklearn.metrics import mean_absolute_error
mae=mean_absolute_error(y_test, y_pred_DTC)
print('mae:',mae)

# mse
from sklearn.metrics import mean_squared_error
mse=mean_squared_error(y_test, y_pred_DTC)
print('mse:',mse)



Accuracy: 0.7637837837837838
mae: 0.23621621621621622
mse: 0.23621621621621622


In [665]:
# guassian naive bayes
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

GNB=GaussianNB()
GNB.fit(X_train, y_train)
y_pred_GNB=GNB.predict(X_test)

accuracy=accuracy_score(y_test, y_pred_GNB)
print('Accuracy:', accuracy)

# mse
from sklearn.metrics import mean_squared_error
mse_GNB=mean_squared_error(y_test, y_pred_GNB)
print('Mean Squared Error:', mse_GNB)

# mae
from sklearn.metrics import mean_absolute_error
mae_GNB=mean_absolute_error(y_test, y_pred_GNB)
print('Mean Absolute Error:', mae_GNB)



Accuracy: 0.7612612612612613
Mean Squared Error: 0.23873873873873874
Mean Absolute Error: 0.23873873873873874


In [666]:
# svc
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

SVM=SVC(gamma='auto')
SVM.fit(X_train, y_train)
y_pred_SVC=SVM.predict(X_test)
accuracy=accuracy_score(y_test, y_pred_SVC)
print('Accuracy:', accuracy)

# mse
from sklearn.metrics import mean_squared_error
mse_SVC=mean_squared_error(y_test, y_pred_SVC)
print('Mean Squared Error:', mse_SVC)


# mae
from sklearn.metrics import mean_absolute_error
mae_SVC=mean_absolute_error(y_test, y_pred_SVC)
print('Mean Absolute Error:', mae_SVC)



Accuracy: 0.8356756756756757
Mean Squared Error: 0.1643243243243243
Mean Absolute Error: 0.1643243243243243
